<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")


## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
import hopsworks

host = "c.app.hopsworks.ai"
project_name = "ML_jacopoda"  # Replace with your project name
api_key = os.environ.get('HOPSWORKS_API_KEY')

try:
    conn = hopsworks.connection(host=host, project=project_name, api_key_value=api_key)
    print(f"Connected to project: {project_name}")
    secrets = hopsworks.get_secrets_api()
    #secrets = conn.get_secrets_api()
    print("Secrets API connected successfully.")
except Exception as e:
    print(f"Error connecting to project: {e}")


2024-12-12 18:42:56,837 INFO: Initializing external client
2024-12-12 18:42:56,837 INFO: Base URL: https://c.app.hopsworks.ai:443
Error connecting to project: expected str, bytes or os.PathLike object, not NoneType


In [4]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = 

with open('data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
print(f"Project Name: {project.name}")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()
#secrets = util.secrets_api("ML_jacopoda")

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
2024-12-12 18:43:40,386 INFO: Initializing external client
2024-12-12 18:43:40,386 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-12 18:43:41,773 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164438
Project Name: ML_jacopoda


'{"country": "italy", "city": "serso", "street": "Via dei Caduti", "aqicn_url": "https://api.waqi.info/feed/A179560", "latitude": 46.07, "longitude": 11.25}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [5]:
# Retrieve feature groups
air_quality_roll_fg = fs.get_feature_group(
    name='air_quality_roll',
    version=1,
)

weather_roll_fg = fs.get_feature_group(
    name='weather_roll',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [6]:
df_aqi= air_quality_roll_fg.read()
df_aqi = df_aqi.sort_values(by='date', ascending=True)
df_aqi

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.45s) 


date       pm25  pm25_3day_roll_mean country   city  \
637  2020-11-10 00:00:00+00:00   5.030000             7.304875   italy  serso   
315  2020-11-12 00:00:00+00:00  20.040001             7.304875   italy  serso   
334  2020-11-13 00:00:00+00:00  20.299999            15.123333   italy  serso   
1337 2020-11-14 00:00:00+00:00  21.990000            20.776667   italy  serso   
1293 2020-11-15 00:00:00+00:00  19.230000            20.506666   italy  serso   
...                        ...        ...                  ...     ...    ...   
1399 2024-11-21 00:00:00+00:00  44.000000            24.666667   italy  serso   
1400 2024-11-22 00:00:00+00:00  12.000000            25.666667   italy  serso   
1401 2024-11-23 00:00:00+00:00  36.000000            30.666667   italy  serso   
1402 2024-11-24 00:00:00+00:00  27.000000            25.000000   italy  serso   
1403 2024-11-25 00:00:00+00:00  37.000000            33.333333   italy  serso   

              street                                 url  
637   Via dei Caduti  https://api.waqi.info/feed/A179560  
315   Via dei Caduti  https://api.waqi.info/feed/A179560  
334   Via dei Caduti  https://api.waqi.info/feed/A179560  
1337  Via dei Caduti  https://api.waqi.info/feed/A179560  
1293  Via dei Caduti  https://api.waqi.info/feed/A179560  
...              ...                                 ...  
1399  Via dei Caduti  https://api.waqi.info/feed/A179560  
1400  Via dei Caduti  https://api.waqi.info/feed/A179560  
1401  Via dei Caduti  https://api.waqi.info/feed/A179560  
1402  Via dei Caduti  https://api.waqi.info/feed/A179560  
1403  Via dei Caduti  https://api.waqi.info/feed/A179560  

[1404 rows x 7 columns]

In [7]:
import requests
import pandas as pd

aq_today_df_roll = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
# Extract the current value (today_value) as a scalar
today_value = aq_today_df_roll["pm25"].iloc[0]  # Use .iloc[0] to get the scalar value
# Extract the last two values (past_2_d) as a list or scalars
past_2_d = df_aqi["pm25"].tail(2).tolist()  # Convert to a list for easier indexing
# Calculate the rolling average using scalar values
roll_aq = (today_value + past_2_d[0] + past_2_d[1]) / 3
#print(roll_aq)

aq_today_df_roll.insert(loc=1, column='pm25_3day_roll_mean', value=roll_aq)

aq_today_df_roll


pm25  pm25_3day_roll_mean country   city          street       date  \
0  28.0            30.666667   italy  serso  Via dei Caduti 2024-12-12   

                                  url  
0  https://api.waqi.info/feed/A179560

In [8]:
aq_today_df_roll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   pm25                 1 non-null      float32       
 1   pm25_3day_roll_mean  1 non-null      float64       
 2   country              1 non-null      object        
 3   city                 1 non-null      object        
 4   street               1 non-null      object        
 5   date                 1 non-null      datetime64[ns]
 6   url                  1 non-null      object        
dtypes: datetime64[ns](1), float32(1), float64(1), object(4)
memory usage: 180.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [9]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 45.75°N 11.5°E
Elevation 505.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


date  temperature_2m_mean  precipitation_sum  wind_speed_10m_max  \
0 2024-12-12                 4.80                0.0            2.620839   
1 2024-12-13                 4.90                0.0            2.968636   
2 2024-12-14                 5.35                0.0            3.415260   
3 2024-12-15                 6.55                0.0            1.800000   
4 2024-12-16                 7.15                0.0            3.600000   
5 2024-12-17                 4.90                0.0            4.024922   
6 2024-12-18                 6.30                0.0            2.160000   
7 2024-12-19                 3.00                0.0            2.811690   
8 2024-12-20                 7.20                0.1            1.484318   
9 2024-12-21                 5.70                0.0            1.938659   

   wind_direction_10m_dominant   city  
0                   164.054535  serso  
1                   165.963730  serso  
2                   161.564957  serso  
3                   143.130020  serso  
4                   143.130020  serso  
5                   169.695206  serso  
6                   180.000000  serso  
7                   140.194473  serso  
8                   255.963730  serso  
9                   158.198532  serso

In [10]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [11]:
# Insert new data
air_quality_roll_fg.insert(aq_today_df_roll)

2024-12-12 18:44:42,527 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164438/fs/1155141/fg/1355088


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: air_quality_roll_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164438/jobs/named/air_quality_roll_1_offline_fg_materialization/executions
2024-12-12 18:44:56,756 INFO: Materialisation job was not scheduled.


(Job('air_quality_roll_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 678922
         }
       },
       "result": {
         "observed_value": 28.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-12T05:44:42.000527Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_

In [12]:
df_aqi= air_quality_roll_fg.read()
df_aqi

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.18s) 


date   pm25  pm25_3day_roll_mean country   city  \
0    2021-05-31 00:00:00+00:00   2.06             1.986667   italy  serso   
1    2022-09-06 00:00:00+00:00   6.72             4.896667   italy  serso   
2    2023-06-17 00:00:00+00:00   5.07             4.123333   italy  serso   
3    2022-01-08 00:00:00+00:00   8.49             5.290000   italy  serso   
4    2021-04-06 00:00:00+00:00   0.85             2.120000   italy  serso   
...                        ...    ...                  ...     ...    ...   
1399 2024-11-21 00:00:00+00:00  44.00            24.666667   italy  serso   
1400 2024-11-22 00:00:00+00:00  12.00            25.666667   italy  serso   
1401 2024-11-23 00:00:00+00:00  36.00            30.666667   italy  serso   
1402 2024-11-24 00:00:00+00:00  27.00            25.000000   italy  serso   
1403 2024-11-25 00:00:00+00:00  37.00            33.333333   italy  serso   

              street                                 url  
0     Via dei Caduti  https://api.waqi.info/feed/A179560  
1     Via dei Caduti  https://api.waqi.info/feed/A179560  
2     Via dei Caduti  https://api.waqi.info/feed/A179560  
3     Via dei Caduti  https://api.waqi.info/feed/A179560  
4     Via dei Caduti  https://api.waqi.info/feed/A179560  
...              ...                                 ...  
1399  Via dei Caduti  https://api.waqi.info/feed/A179560  
1400  Via dei Caduti  https://api.waqi.info/feed/A179560  
1401  Via dei Caduti  https://api.waqi.info/feed/A179560  
1402  Via dei Caduti  https://api.waqi.info/feed/A179560  
1403  Via dei Caduti  https://api.waqi.info/feed/A179560  

[1404 rows x 7 columns]

In [13]:
# Insert new data
weather_roll_fg.insert(daily_df)

2024-12-12 18:45:02,999 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164438/fs/1155141/fg/1354145


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_roll_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164438/jobs/named/weather_roll_1_offline_fg_materialization/executions


(Job('weather_roll_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677986
         }
       },
       "result": {
         "observed_value": 1.4843180179595947,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-12T05:45:02.000999Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
